In [ ]:
import time
from time import sleep
from random import randint
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from requests import Session
import psycopg2

In [ ]:
def create_incognito_session():
    session = Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
    })
    return session

base_url = "https://www.magicbricks.com/mbsrp/propertySearch.html?editSearch=Y&category=R&propertyType=10001,10017,10002,10003,10021,10022,10020&bedrooms=11700,11701,11702,11703&city=2481&page={page_number}&groupstart=90&offset=0&maxOffset=100&sortBy=premiumRecent&pType=10001,10017,10002,10003,10021,10022,10020&isNRI=N&showPrimePropsinFixedSlotsSEO=N&multiLang=en"

num_pages = 150  

session = create_incognito_session()

conn = psycopg2.connect(
    host="localhost",
    port="XXXX",
    dbname="XXXX",
    user="XXXX",
    password="XXXX"
)

cur = conn.cursor()

cur.execute("""
    CREATE TABLE IF NOT EXISTS magicbricks_listings (
        id SERIAL PRIMARY KEY,
        Price VARCHAR(255),
        Landmarks TEXT,
        Min_Price VARCHAR(255),
        Max_Price VARCHAR(255),
        Carpet_Area VARCHAR(255),
        Price_Sqft VARCHAR(255),
        City VARCHAR(255),
        Unit VARCHAR(50),
        Bedroom VARCHAR(50),
        Title TEXT,
        Tenants_Preference TEXT,
        Project_Name VARCHAR(255)
    );
""")

insert_query = """
    INSERT INTO magicbricks_listings (Price, Landmarks, Min_Price, Max_Price, Carpet_Area, Price_Sqft, City, Unit, Bedroom, Title, Tenants_Preference, Project_Name)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

for page in range(1, num_pages + 1):
    url = base_url.format(page_number=page)
    
    response = session.get(url)
    
    if response.status_code == 200:
        data = response.json()  
        
        if 'resultList' in data:
            for property in data['resultList']:
                record = (
                    property.get("price"),
                    "; ".join(property.get("landmarkDetails", [])),  
                    property.get("minPrice"),
                    property.get("maxPrice"),
                    property.get("caSqFt"),
                    property.get("sqFtPrice"),
                    property.get("ctName"),
                    property.get("coverAreaUnitD"),
                    property.get("bedroomD"),
                    property.get("auto_desc"),
                    property.get("tenantsPreference"),
                    property.get("prjname"),
                )
                
                cur.execute(insert_query, record)
        else:
            print(f"No results found on page {page}")
            break  
    else:
        print(f"Failed to retrieve data on page {page}, Status code: {response.status_code}")
        break  
    
    sleep_time = randint(2, 5)  
    print(f"Sleeping for {sleep_time} seconds...")
    time.sleep(sleep_time)

conn.commit()

cur.close()
conn.close()

print("Data has been written to the PostgreSQL database.")